In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input
from scipy.linalg import sqrtm
from PIL import Image


In [2]:
def load_image(image_path,dim1,dim2):
    img= Image.open(image_path).resize((dim1,dim2)).convert("RGB")
    return np.array(img)

In [9]:
from skimage.transform import resize

def resize_images(images):
    resized_images = []
    for img in images:
        resized_img = resize(img, (299, 299, 3), anti_aliasing=True)
        resized_images.append(resized_img)
    return np.array(resized_images)

In [3]:
# Load pre-trained InceptionV3 model
inception_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')

# Function to extract features from images using InceptionV3
def extract_features(images):
    images = preprocess_input(images)
    features = inception_model.predict(images)
    return features

In [13]:

def calculate_fid(real_features, generated_features):
    mu_real = np.mean(real_features, axis=0)
    mu_generated = np.mean(generated_features, axis=0)
    sigma_real = np.cov(real_features, rowvar=False)
    sigma_generated = np.cov(generated_features, rowvar=False)
    
    # Calculate sum of squared differences between means
    ssdiff = np.sum((mu_real - mu_generated)**2)
    
    # Calculate square root of product of covariances
    covmean = sqrtm(sigma_real.dot(sigma_generated))
    
    # Handle complex numbers, if any
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    # Calculate FID score
    fid = ssdiff + np.trace(sigma_real + sigma_generated - 2.0 * covmean)

    if fid < 0:
        fid = -1*fid
    
    return fid

In [10]:
real_images = "d:/Study Materials/4-2/paint like any painter/Image_data/photo_jpg/1.jpg"
generated_images = "d:/Study Materials/4-2/paint like any painter/monet_jpggenerated/1.jpg"
cycle_gan = "d:/Study Materials/4-2/paint like any painter/generated_images/1.jpg"
# convert to numpy array
dim = 299
real_images = load_image(real_images,dim,dim)
generated_images = load_image(generated_images,dim,dim)
cycle_gan = load_image(cycle_gan,dim,dim)

# Resize images
real_images = resize_images(real_images)
generated_images = resize_images(generated_images)
cycle_gan = resize_images(cycle_gan)


In [11]:
# Calculate FID score
real_features= extract_features(real_images)
generated_features = extract_features(generated_images)
cycle_gan = extract_features(cycle_gan)
print(real_features.shape)
print(generated_features.shape)
print(cycle_gan.shape)


10/10 [==============================] - 10s 994ms/step
(299, 2048)
(299, 2048)
(299, 2048)


In [14]:
# reshape the feature arrays
fid_score_real_vgg  = calculate_fid(real_features, generated_features)
cycle_gan_fid_score = calculate_fid(real_features, cycle_gan)



In [15]:
print("Real vs VGG FID Score:       ", fid_score_real_vgg)
print("REAL vs Cycle GAN FID Score: ", cycle_gan_fid_score)

Real vs VGG FID Score:        1.8788340662190666e+111
REAL vs Cycle GAN FID Score:  5.96787883691744e+87
